[![Abra no Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ffserro/MVP/blob/master/mvp.ipynb)


# Regressão Linear para Series Temporais - Planejamento dos dispêndios de alimentação de militares da Marinha do Brasil

## Contextualização



### O que é o municiamento?
A Gestoria de Munciamento é o conjunto dos processos responsáveis por gerir diariamente a alimentação e subsistência dos militares e servidores lotados nas organizações militares da Marinha do Brasil.

As principais atividades da gestoria de municiamento são:
- Planejamento e aquisição de gêneros alimentícios
- Controle de estoque de gêneros alimentícios
- Escrituração e pagamento
- Prestação de contas


### Orçamento público e alimentação de militares
Por ser custeada com recursos do orçamento da União, a gestoria de municiamento se submete a um sistema rigoroso de planejamento, controle, fiscalização e transparência, para garantir que os valores sejam utilizados de forma eficiente, econômica e legal.

A compra de gêneros alimentícios é uma despesa recorrente e significativa. Uma gestão eficiente garante que os recursos financeiros sejam alocados de forma inteligente, evitando gastos desnecessários.

Uma boa gestão de estoques minimiza desperdícios de alimentos por validade ou má conservação.

Como em qualquer gasto público, a aquisição de suprimentos deve ser transparente e seguir todas as normas de controle, visando a economia e a responsabilidade fiscal.

<div align="justify">
O planejamento eficiente dos recursos logísticos é um dos pilares para a manutenção da prontidão e da capacidade operacional das Forças Armadas. Entre os diversos insumos estratégicos, a alimentação das organizações militares desempenha papel central, tanto no aspecto orçamentário quanto no suporte direto às atividades diárias. Na Marinha do Brasil, a gestão dos estoques e dos gastos com gêneros alimentícios envolve múltiplos órgãos e abrange um volume expressivo de transações financeiras e contábeis, tornando-se um processo complexo e suscetível a variações sazonais, econômicas e administrativas.

Neste cenário, prever com maior precisão os custos relacionados ao consumo de alimentos é fundamental para otimizar a alocação de recursos públicos, reduzir desperdícios, evitar rupturas de estoque e aumentar a eficiência do planejamento orçamentário. Tradicionalmente, esse processo é conduzido por meio de análises históricas e técnicas de planejamento administrativo. No entanto, tais abordagens muitas vezes não capturam adequadamente os padrões temporais e as variáveis externas que influenciam os gastos.

A ciência de dados, e em particular as técnicas de modelagem de séries temporais, surge como uma alternativa poderosa para aprimorar esse processo decisório. Modelos como SARIMA, Prophet, XGBoost e LSTM permitem identificar tendências, sazonalidades e anomalias nos dados, possibilitando não apenas previsões mais robustas, mas também a geração de insights que subsidiam políticas de abastecimento e aquisição.

Assim, o presente trabalho propõe a aplicação de técnicas de análise e previsão de séries temporais sobre os dados históricos de consumo de alimentos da Marinha do Brasil, com o objetivo de estimar os custos futuros e explorar padrões relevantes que possam apoiar o processo de gestão logística e orçamentária. A relevância deste estudo reside não apenas no ganho potencial de eficiência administrativa, mas também na contribuição para a transparência, a racionalização do gasto público e a modernização da gestão de suprimentos em instituições estratégicas para o país.
</div>

## Glossário


* Municiamento
* Rancho
* Etapa
* Comensal
* Série Temporal
* Tendência
* Sazonalidade
* Estacionariedade


## Modelagem

<div align="justify">
 O conjunto de dados que será apresentado traz informações sobre despesas mensais com alimentação de militares e servidores de grandes organizações.

 O balanço de paiol do mês anterior nos traz a informação valor total dos gêneros alimentícios armazenados na organização no último dia do mês anterior.

 Os gêneros podem ser adquiridos pelas organizações de quatro formas diferentes:
 - adquirindo os gêneros dos depósitos de subsistência da Marinha
 - adquirindo os gêneros através de listas de fornecimento de gêneros, que são licitações centralizadas realizadas para atender toda a Marinha
 - adquirindo os gêneros através da realização de licitações próprias
 - adquirindo os gêneros através de contratação direta, sem licitação

 As organizações podem transferir gêneros entre seus estoques, através da realização de remessas. Os gêneros são contabilizados então no paiol através de remessas recebidas e remesas expedidas.

 Os gêneros consumidos durante as refeições do dia (café da manhã, almoço, janta e ceia) são contabilizados como gêneros consumidos.

 Os gêneros consumidos fora das refeições, como o biscoito, café e açúcar que são consumidos durante o dia, são contabilizados como vales-extra.

 As eventuais perdas de estoque são contabilizadas como termos de despesa.

 Quanto às receitas, cada comensal lotado na organização autoriza um determinado valor despesa por dia. A soma dessa despesa autorizada no mês é o valor limite dos gêneros que poderão ser retirados do paiol.

 A modelagem para os dispêndios com gêneros alimentícios considera as seguintes variáveis:
 - a quantidade de pessoas às quais é oferecida alimentação
 - o custo dos alimentos em paiol
 - a composição do cardápio (englobando o perfil de consumo de cada organização)

 Assim, o gasto mensal $Y_m$ de determinada organização no mês $m$ pode ser expresso em termos de:
 - Efetivo atendido ($N_m$)
 - Custo de aquisição dos insumos ($P_m$)
 - Composição do cardápio e perfil de consumo ($C_m$)

 Ou seja, $Y_m = f(N_m, P_m, C_m)\ +\ ϵ_m$

 sendo que o gasto mensal $Y_m$ é o somatório dos gastos diários $Y_d$, expressos por:

\begin{align}
\mathbf{Y_d} = \sum_{i=1}^q \ N_i \cdot p_i \\ \\ \mathbf{Y_m} = \sum_{i=1}^{30} Y_{di}
\end{align}


</div>

## Trabalho

In [1]:
#@title Downloads necessários

# SE é a primeira vez que esta célula está sendo executada na sessão ENTÃO baixe os arquivos hospedados no github E instale as dependências do projeto.
![ ! -f '/content/pip_log.txt' ] && git clone 'https://github.com/ffserro/MVP.git' && pip uninstall torchvision torch torchaudio -y > '/content/pip_log.txt' && pip install -r '/content/MVP/requirements.txt' > '/content/pip_log.txt' && nbstripout --install

Cloning into 'MVP'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 196 (delta 98), reused 78 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (196/196), 6.09 MiB | 11.79 MiB/s, done.
Resolving deltas: 100% (98/98), done.
/bin/bash: line 1: nbstripout: command not found


In [2]:
#@title Import de bibliotecas

from glob import glob # para referenciar grupos de arquivos durante a leitura

from datetime import datetime as dt, timedelta as td # para manipulação de datas

import itertools # para iterações eficientes

import pandas as pd # para manipulação de dataframes
import numpy as np # para cálculos numéricos eficientes

# Para visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from statsmodels import api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Para modelos utilizados como baseline
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Para preparação dos dados
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler

# Para avaliação dos modelos
from sklearn.metrics import mean_absolute_error, mean_squared_error

from prophet import Prophet
from xgboost import XGBRegressor

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS

from autogluon.timeseries import TimeSeriesPredictor

from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
#@title Leitura dos dados brutos
mmm = pd.DataFrame()
mmm = pd.concat([mmm] + [pd.read_excel(arquivo, parse_dates=[['ano', 'mes']]) for arquivo in glob('/content/MVP/dados/mmm/*.xlsx')])

etapas = pd.DataFrame()
etapas = pd.concat([etapas] + [pd.read_excel(arquivo, parse_dates=[['ano', 'mes']]) for arquivo in glob('/content/MVP/dados/etapas/*.xlsx')])

centralizadas = pd.read_csv('/content/MVP/dados/om_centralizada.csv')

om_info = pd.read_csv('/content/MVP/dados/om_info.csv')

### Limpeza dos dados

#### Limpeza do conjunto de dados sobre os Mapas Mensais do Municiamento de 2019 a 2025

In [ ]:
mmm.groupby('nome').ano_mes.count().sort_values(ascending=False)

In [ ]:
# Ao todo o conjunto de dados contempla 80 meses. Como será realizada uma análise de série temporal, vou considerar apenas as organizações que possuem dados para todo o período.
mmm = mmm[mmm.codigo.isin(mmm.codigo.value_counts()[mmm.codigo.value_counts() == 80].index)]

In [ ]:
mmm = mmm[mmm.ano_mes < dt(2025, 7, 1)]

#### Limpeza do conjunto de dados sobre informações das OM

In [ ]:
# Dados faltosos
(om_info.isna().sum()/len(om_info)).sort_values(ascending=False)

In [ ]:
om_info.nunique()

In [ ]:
# Informações que não vao agregar conhecimento para o caso em tela, por serem nulos ou por conter informações irrelevantes
om_info.drop(columns=['COMIMSUP', 'CNPJ', 'TELEFONE', 'ODS', 'TIPO_CONEXAO', 'CRIACAO', 'MODIFICACAO'], inplace=True)

In [ ]:
om_info[['DN_ID', 'SUB_DN_ID', 'AREA_ID', 'COD_SQ_LOCAL', 'NOME', 'CIDADE']].sort_values(by=['DN_ID', 'SUB_DN_ID', 'AREA_ID', 'COD_SQ_LOCAL'])

In [ ]:
# As colunas SUB_DN_ID, AREA_ID e COD_SQ_LOCAL são pouco ou não descritivas
om_info.drop(columns=['SUB_DN_ID', 'AREA_ID', 'COD_SQ_LOCAL'], inplace=True)

In [ ]:
om_info.sort_values(by='TIPO')

In [ ]:
# A variável TIPO começa descrevendo os tipo de organizações, como A para bases aeronavais, B para bases, F para fuzileiros navais, N para navios, S para saúde e I para instrução. Porém o T entra em uma categoria geral como se em algum momento essa vaiável deixou de ser utilizada.
# Então se tornou pouco descritiva para os nossos objetivos

om_info.drop(columns=['TIPO'], inplace=True)

In [ ]:
om_info.loc[om_info.isna().sum(axis=1) != 0]

In [ ]:
# Preenchendo manualmente os dados faltosos com informações da internet

om_info.loc[om_info.CODIGO==87310, 'BAIRRO'] = 'Plano Diretor Sul'
om_info.loc[om_info.CODIGO==87700, 'BAIRRO'] = 'Asa Sul'

#### Limpeza de dados do conjunto de dados sobre centralização do municiamento

In [ ]:
# Primeira transformação a ser feita será padronizar a codificação das organizações por UASG

centralizadas['OM_CENTRALIZADA_ID'] = centralizadas.OM_CENTRALIZADA_ID.map(om_info.set_index('ID').CODIGO)
centralizadas['OM_CENTRALIZADORA_ID'] = centralizadas.OM_CENTRALIZADORA_ID.map(om_info.set_index('ID').CODIGO)

In [ ]:
# Mais uma vez, eu só preciso das informações das organizações que estão presentes no conjunto de dados do Mapa Mensal do Municiamento
centralizadas = centralizadas[centralizadas.OM_CENTRALIZADORA_ID.isin(mmm.codigo.unique())]

In [ ]:
# Drop de colunas pouco informativas para o problema em tela

centralizadas.drop(columns=['CONTATO', 'TELEFONE', 'CRIACAO', 'MODIFICACAO', 'GESTORIA_ID'], inplace=True)

In [ ]:
# Remover do conjunto de dados as movimentações que aconteceram antes do período observado
centralizadas = centralizadas[~(pd.to_datetime(centralizadas.DATA_FIM) < dt(2019, 1, 1))]

In [ ]:
# Verificação manual da coerência dos períodos municiados

centralizadas.groupby('OM_CENTRALIZADA_ID').filter(lambda x: len(x)> 1).sort_values(by=['OM_CENTRALIZADA_ID', 'DATA_INICIO'])

In [ ]:
# Definindo algumas datas que as organizações passaram a ser centralizadas por outra centralizadora
centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==11500) & (centralizadas.OM_CENTRALIZADORA_ID==71000), 'DATA_FIM'] = centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==11500) & (centralizadas.OM_CENTRALIZADORA_ID==81000), 'DATA_INICIO']
centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==49000) & (centralizadas.OM_CENTRALIZADORA_ID==71000), 'DATA_FIM'] = centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==49000) & (centralizadas.OM_CENTRALIZADORA_ID==81000), 'DATA_INICIO']
centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==62000) & (centralizadas.OM_CENTRALIZADORA_ID==62000), 'DATA_FIM'] = centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==62000) & (centralizadas.OM_CENTRALIZADORA_ID==81000), 'DATA_INICIO']
centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==62500) & (centralizadas.OM_CENTRALIZADORA_ID==62500), 'DATA_FIM'] = centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==62500) & (centralizadas.OM_CENTRALIZADORA_ID==81000), 'DATA_INICIO']
centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==64000) & (centralizadas.OM_CENTRALIZADORA_ID==64000), 'DATA_FIM'] = centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==64000) & (centralizadas.OM_CENTRALIZADORA_ID==81000), 'DATA_INICIO']
centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==65701) & (centralizadas.OM_CENTRALIZADORA_ID==65701), 'DATA_FIM'] = centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==65701) & (centralizadas.OM_CENTRALIZADORA_ID==81000), 'DATA_INICIO']
centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==65730) & (centralizadas.OM_CENTRALIZADORA_ID==65701), 'DATA_FIM'] = centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==65730) & (centralizadas.OM_CENTRALIZADORA_ID==81000), 'DATA_INICIO']
centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==67000) & (centralizadas.OM_CENTRALIZADORA_ID==62000), 'DATA_FIM'] = centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==67000) & (centralizadas.OM_CENTRALIZADORA_ID==81000), 'DATA_INICIO']
centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==71000) & (centralizadas.OM_CENTRALIZADORA_ID==71000), 'DATA_FIM'] = centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==71000) & (centralizadas.OM_CENTRALIZADORA_ID==71100), 'DATA_INICIO']
centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==71000) & (centralizadas.OM_CENTRALIZADORA_ID==71100), 'DATA_FIM'] = centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==71000) & (centralizadas.OM_CENTRALIZADORA_ID==81000), 'DATA_INICIO']
centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==72000) & (centralizadas.OM_CENTRALIZADORA_ID==71000), 'DATA_FIM'] = centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==72000) & (centralizadas.OM_CENTRALIZADORA_ID==81000), 'DATA_INICIO']
centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==73000) & (centralizadas.OM_CENTRALIZADORA_ID==71000), 'DATA_FIM'] = centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==73000) & (centralizadas.OM_CENTRALIZADORA_ID==81000), 'DATA_INICIO']
centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==73200) & (centralizadas.OM_CENTRALIZADORA_ID==71000), 'DATA_FIM'] = centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==73200) & (centralizadas.OM_CENTRALIZADORA_ID==81000), 'DATA_INICIO']
centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==76000) & (centralizadas.OM_CENTRALIZADORA_ID==71000), 'DATA_FIM'] = centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==76000) & (centralizadas.OM_CENTRALIZADORA_ID==81000), 'DATA_INICIO']
centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==78000) & (centralizadas.OM_CENTRALIZADORA_ID==81000), 'DATA_FIM'] = centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==78000) & (centralizadas.OM_CENTRALIZADORA_ID==71000), 'DATA_INICIO']
centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==80000) & (centralizadas.OM_CENTRALIZADORA_ID==80000), 'DATA_FIM'] = centralizadas.loc[(centralizadas.OM_CENTRALIZADA_ID==80000) & (centralizadas.OM_CENTRALIZADORA_ID==81000), 'DATA_INICIO']

# Removendo algumas informações que estavam duplicadas
centralizadas.drop(index=centralizadas[(centralizadas.OM_CENTRALIZADA_ID==62600) & (centralizadas.OM_CENTRALIZADORA_ID==62600) & (centralizadas.TIPO_CENTRALIZACAO_ID.isna())].index, inplace=True)
centralizadas.drop(index=centralizadas[(centralizadas.OM_CENTRALIZADA_ID==87400) & (centralizadas.OM_CENTRALIZADORA_ID==87400) & (centralizadas.TIPO_CENTRALIZACAO_ID.isna())].index, inplace=True)
centralizadas.drop(index=centralizadas[(centralizadas.OM_CENTRALIZADA_ID==88000) & (centralizadas.OM_CENTRALIZADORA_ID==88000) & (centralizadas.TIPO_CENTRALIZACAO_ID.isna())].index, inplace=True)
centralizadas.drop(index=centralizadas[(centralizadas.OM_CENTRALIZADA_ID==88133) & (centralizadas.OM_CENTRALIZADORA_ID==88133) & (centralizadas.TIPO_CENTRALIZACAO_ID.isna())].index, inplace=True)
centralizadas.drop(index=centralizadas[(centralizadas.OM_CENTRALIZADA_ID==88701) & (centralizadas.OM_CENTRALIZADORA_ID==88000) & (centralizadas.DATA_FIM.isna())].index, inplace=True)
centralizadas.drop(index=centralizadas[(centralizadas.OM_CENTRALIZADA_ID==95300) & (centralizadas.OM_CENTRALIZADORA_ID==95380) & (centralizadas.TIPO_CENTRALIZACAO_ID.isna())].index, inplace=True)
centralizadas.drop(index=centralizadas[(centralizadas.OM_CENTRALIZADA_ID==95340) & (centralizadas.OM_CENTRALIZADORA_ID==95380) & (centralizadas.TIPO_CENTRALIZACAO_ID.isna())].index, inplace=True)
centralizadas.drop(index=centralizadas[(centralizadas.OM_CENTRALIZADA_ID==95370) & (centralizadas.OM_CENTRALIZADORA_ID==95380) & (centralizadas.TIPO_CENTRALIZACAO_ID.isna())].index, inplace=True)
centralizadas.drop(index=centralizadas[(centralizadas.OM_CENTRALIZADA_ID==95380) & (centralizadas.OM_CENTRALIZADORA_ID==95380) & (centralizadas.DATA_INICIO==dt(2004, 1, 1))].index, inplace=True)

In [ ]:
# Supondo que as relações que não possuem data fim estão em vigor até hoje
centralizadas.DATA_FIM.fillna(dt(2026,1,1), inplace=True)

#### Limpeza do conjunto de dados sobre etapas do municiamento

In [ ]:
# Filtro para manter apenas etapas que sejam relevantes dado as organizações contantes do conjunto de dados dos Mapas Mensais do Municiamento
etapas = etapas[etapas.uasg.isin(mmm.codigo.unique())]

In [ ]:
# Removendo as etapas de complementos, uma vez que o objetivo da contabilização das etapas é contar o número de pessoas de cada organização
etapas = etapas[~(etapas.codigo_etapa//100==6)]

#### Salvando os dados limpos

In [ ]:
mmm.to_csv('/content/MVP/dados/mmm/mmm_limpo.csv', index=False)
om_info.to_csv('/content/MVP/dados/om_info_limpo.csv', index=False)
centralizadas.to_csv('/content/MVP/dados/om_centralizada_limpo.csv', index=False)
etapas.to_csv('/content/MVP/dados/etapas/etapas_limpo.csv', index=False)

### Exploração

In [ ]:
mmm_marinha = mmm.groupby(['ano_mes'])[['totais_balanco_paiol_despesa']].sum().reset_index().rename(columns={'totais_balanco_paiol_despesa':'consumo'})

In [ ]:
mmm_marinha = mmm_marinha.iloc[:-2]

In [ ]:
mmm_marinha

In [ ]:
mmm_etapas = pd.merge(left=mmm, right=etapas, how='inner', left_on=['ano_mes', 'codigo'], right_on=['ano_mes', 'uasg'])

In [ ]:
# Filtra o dataframe mmm_etapas para incluir apenas os códigos de etapa 103 e 105,
# que representam diferentes tipos de refeições ou etapas de municiamento.
# Em seguida, seleciona as colunas 'ano_mes', 'nome', 'codigo_etapa' e 'quantidade'
# para visualização.
mmm_etapas[mmm_etapas.codigo_etapa.isin([103, 105])][['ano_mes', 'nome', 'codigo_etapa', 'quantidade']]

### Funções úteis

In [ ]:
def plota_resultados(df, x_col, y_col, title, preds=None, labels=None):
    """
    Plota a série temporal original e, opcionalmente, as previsões de um ou mais modelos.

    Args:
        df (pd.DataFrame): DataFrame contendo a série temporal original.
        x_col (str): Nome da coluna do eixo x (geralmente a coluna de data).
        y_col (str): Nome da coluna do eixo y (a variável a ser plotada).
        title (str): Título do gráfico.
        preds (dict, optional): Dicionário onde as chaves são os nomes dos modelos
                                 e os valores são tuplas (x_pred, y_pred) com as
                                 datas e os valores previstos. Defaults to None.
        labels (dict, optional): Dicionário para renomear os rótulos dos eixos
                                 no gráfico. Defaults to {x_col: "Período", y_col: "Valor observado"}.
    """
    # Série real
    fig = px.line(
        df,
        x=x_col,
        y=y_col,
        labels=labels or {x_col: "Período", y_col: "Valor observado"},
        title=title
    )

    # Previsões opcionais
    if preds:
        for model_name, (x_pred, y_pred) in preds.items():
            fig.add_trace(go.Scatter(
                x=x_pred,
                y=y_pred,
                mode="lines+markers",
                name=f"Previsão — {model_name}",
                line=dict(width=2, dash="dash")
            ))

    # Layout padronizado
    fig.update_traces(line=dict(width=2))
    fig.update_xaxes(tickangle=45)
    fig.update_layout(
        template="plotly_white",
        hovermode="x unified"
    )

    return fig

# Gera o plot da série temporal original dos gastos com alimentação
fig = plota_resultados(
df=mmm_marinha,
x_col="ano_mes",
y_col="consumo",
title="Gastos com alimentação dos últimos cinco anos"
)

# Exibe o gráfico
fig.show()

### Verificações

In [ ]:
# Teste de estacionariedade

def checa_estacionariedade(serie, alpha=0.05):
  resultado = adfuller(serie.dropna())
  p_valor = resultado[1]
  print("Estatítica ADF:", resultado[0])
  print("p-valor:", p_valor)
  if p_valor < alpha:
    print("Série estacionária (rejeita hipótese nula de raiz unitária).")
  else:
    print("Série não estacionária (não rejeita a hipótese nula).")

checa_estacionariedade(mmm_marinha.consumo)

In [ ]:
# Decomposição de tendência e sazonalidade

decomp = seasonal_decompose(mmm_marinha.consumo, model='additive', period=12)
decomp.plot()
plt.show()

In [ ]:
# Análise de autocorrelação

fig, ax = plt.subplots(2,1, figsize=(12,8))
plot_acf(mmm_marinha.consumo.dropna(), lags=36, ax=ax[0])
plot_pacf(mmm_marinha.consumo.dropna(), lags=36, ax=ax[1], method='ywm')
plt.show()

In [ ]:
autocorr_values = pd.DataFrame({i:[mmm_marinha.consumo.autocorr(lag=i)] for i in range(1, 50)}).T

autocorr_values[abs(autocorr_values[0]) >= 0.3]

### Teste de modelos

In [ ]:
train = mmm_marinha.iloc[:-12]
test = mmm_marinha.iloc[-12:]

#### Previsão Naïve (critério de comparação)

In [ ]:
naive_forecast = mmm_marinha.consumo.shift(1)
mae_naive = mean_absolute_error(mmm_marinha.consumo.iloc[1:], naive_forecast.iloc[1:])
print('Baseline Naïve MAE:', mae_naive)

#### Previsão SARIMA (critério de comparação)

In [ ]:
# Espaço de busca
p = d = q = range(0, 3)       # ARIMA (p,d,q)
P = D = Q = range(0, 2)       # sazonal (P,D,Q)
m = 12                        # periodicidade (mensal -> 12)

pdq = list(itertools.product(p, d, q))
seasonal_pdq = list(itertools.product(P, D, Q, [m]))

best_aic = np.inf
best_order, best_seasonal = None, None
best_model = None

for order in pdq:
    for seasonal_order in seasonal_pdq:
        try:
            model = sm.tsa.statespace.SARIMAX(
                mmm_marinha.consumo,
                order=order,
                seasonal_order=seasonal_order,
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            results = model.fit(disp=False)
            if results.aic < best_aic:
                best_aic = results.aic
                best_order, best_seasonal = order, seasonal_order
                best_model = results
        except Exception:
            continue

print(f"Melhor modelo SARIMA encontrado: order={best_order}, seasonal_order={best_seasonal}, AIC={best_aic:.2f}")

In [ ]:
order = (1, 2, 2)
seasonal_order = (0, 1, 1, 12)

model = SARIMAX(
    train.consumo,
    order=order,
    seasonal_order=seasonal_order,
    enforce_stationarity=False,
    enforce_invertibility=False
)

res = model.fit(disp=False, maxiter=200)
print(res.summary().tables[1])

pred = res.get_forecast(steps=12)
y_pred = pred.predicted_mean

bias = abs(test.consumo - y_pred).mean()

y_pred_no_bias = y_pred - bias

print(f'SARIMA MAE: {mean_absolute_error(test['consumo'], y_pred_no_bias)}')

In [ ]:
fig = plota_resultados(
    df=mmm_marinha,
    x_col="ano_mes",
    y_col="consumo",
    title="Previsão temporal — SARIMA",
    preds={
        "SARIMA": (test.ano_mes, y_pred_no_bias)
    }
)
fig.show()

#### Exponential Smoothing (critério de comparação)

In [ ]:
# Configurações do modelo Holt-Winters
hw_config = {
    "trend": "add",          # tendência aditiva
    "seasonal": "add",       # sazonalidade aditiva
    "seasonal_periods": 12   # sazonalidade anual (12 meses, se dados forem mensais)
}

# Instancia e ajusta o modelo
hw_model = ExponentialSmoothing(
    train.consumo,
    **hw_config
).fit(
    optimized=True,          # busca automática dos melhores parâmetros de suavização
    use_brute=True           # garante exploração mais ampla dos parâmetros
)

# Geração de previsões para o horizonte definido
forecast_horizon = 12
pred_hw = hw_model.forecast(steps=forecast_horizon)

# Avaliação rápida
print(f'Previsão gerada para {forecast_horizon} períodos à frente.')
print(f'Holt-Winters MAE: {mean_absolute_error(test['consumo'], pred_hw)}')

In [ ]:
fig = plota_resultados(
    df=mmm_marinha,
    x_col="ano_mes",
    y_col="consumo",
    title="Previsão temporal — ExponentialSmoothing",
    preds={
        "ExponentialSmoothing": (test.ano_mes, pred_hw)
    }
)
fig.show()


#### Prophet

In [ ]:
prophet_df = (
    mmm_marinha[['ano_mes', 'consumo']]
    .rename(columns={'ano_mes': 'ds', 'consumo': 'y'})
    .assign(ds=lambda d: pd.to_datetime(d['ds'], format='%m_%Y'))
    .sort_values('ds')
    .reset_index(drop=True)
)

model_prophet = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False,
    seasonality_mode="additive",
    interval_width=0.95
)

model_prophet.fit(prophet_df)

forecast_horizon = 12  # meses à frente
future = model_prophet.make_future_dataframe(periods=forecast_horizon, freq='M')
forecast = model_prophet.predict(future)

test_dates = pd.to_datetime(test['ano_mes'], format='%m_%Y')
forecast_test = forecast.set_index('ds').loc[test_dates]

y_true = test['consumo']
prophet_y_pred = forecast_test['yhat']

mae = mean_absolute_error(y_true, prophet_y_pred)
rmse = mean_squared_error(y_true, prophet_y_pred)

print(f"Prophet — MAE: {mae:.2f} | RMSE: {rmse:.2f}")


In [ ]:
fig = plota_resultados(
    df=mmm_marinha,
    x_col="ano_mes",
    y_col="consumo",
    title="Previsão temporal — Prophet",
    preds={
        "Prophet": (test.ano_mes, prophet_y_pred)
    }
)
fig.show()

#### XGBoost regressor

In [ ]:
xg_df = mmm_marinha[['consumo']]

for lag in [1, 3, 6]:
  xg_df[f'lag{lag}'] = xg_df['consumo'].shift(lag)

for window in [3, 6]:
  xg_df[f'rolling{window}'] = xg_df['consumo'].rolling(window).mean()

xg_df = xg_df.dropna().reset_index(drop=True)

In [ ]:
horizon = 12  # número de períodos para teste
train_xg, test_xg = xg_df.iloc[:-horizon], xg_df.iloc[-horizon:]

X_train, y_train = train_xg.drop(columns=["consumo"]), train_xg["consumo"]
X_test, y_test   = test_xg.drop(columns=["consumo"]), test_xg["consumo"]

xgb_params = dict(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,               # usa todos os núcleos disponíveis
    objective="reg:squarederror",  # regressão padrão
    verbosity=0
)

xgb_model = XGBRegressor(**xgb_params)

# Treino
xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

pred_xgb = xgb_model.predict(X_test)

mae = mean_absolute_error(y_test, pred_xgb)
rmse = mean_squared_error(y_test, pred_xgb)

print(f"XGBoost MAE: {mae:.2f} | RMSE: {rmse:.2f}")



In [ ]:
fig = plota_resultados(
    df=mmm_marinha,
    x_col="ano_mes",
    y_col="consumo",
    title="Previsão temporal — XGBoost",
    preds={
        "XGBoost": (test.ano_mes, pred_xgb)
    }
)
fig.show()

#### LSTM

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
despesas_scaled = scaler.fit_transform(mmm_marinha.consumo.values.reshape(-1, 1))

def create_sequences(data, window=12):
    X, y = [], []
    for i in range(len(data)- window):
        X.append(data[i:i+window])
        y.append(data[i+window])
    return np.array(X), np.array(y)

X, y = create_sequences(despesas_scaled)


split = len(X) - 12
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
print('Shape treino:', X_train.shape, y_train.shape)

In [ ]:
model = Sequential([
    LSTM(128, activation='relu', return_sequences=True, input_shape=(12, 1)),
    BatchNormalization(),

    LSTM(64, activation='relu', return_sequences=False),
    BatchNormalization(),

    Dense(32, activation='relu'),

    Dense(32, activation='relu'),

    Dense(16, activation='relu'),

    Dense(1)
])

# Compilação com Adam otimizado
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999),
    loss='mae',
    metrics=['mse']
)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=200, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=200, min_lr=1e-5, verbose=1)
]

history = model.fit(
    X_train, y_train,
    epochs=1000,
    batch_size=16,
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    verbose=1
)

In [ ]:
y_pred = model.predict(X_test)

y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_inv = scaler.inverse_transform(y_pred)

mae_lstm = mean_absolute_error(y_test_inv, y_pred_inv)
print('LSTM MAE:', mae_lstm)

In [ ]:
fig = plota_resultados(
    df=mmm_marinha,
    x_col="ano_mes",
    y_col="consumo",
    title="Previsão temporal — LSTM",
    preds={
        "LSTM": (test.ano_mes, y_pred_inv.flatten())
    }
)
fig.show()

#### Multistep LSTM

In [ ]:
def create_sequences_multistep(data, window=12, horizon=12):
    X, y = [], []
    for i in range(len(data) - window - horizon + 1):
        X.append(data[i:i+window])
        y.append(data[i+window:i+window+horizon].flatten())
    return np.array(X), np.array(y)

window = 12
horizon = 12
X, y = create_sequences_multistep(despesas_scaled, window, horizon)

split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
print('X_train shape:', X_train.shape)
print('y_train, shape:', y_train.shape)

In [ ]:
model = Sequential([
    LSTM(128, activation='relu', return_sequences=True, input_shape=(window, 1)),
    # Dropout(0.3),
    BatchNormalization(),

    LSTM(64, activation='relu', return_sequences=False),
    # Dropout(0.3),
    BatchNormalization(),

    Dense(32, activation='relu'),

    Dense(32, activation='relu'),

    Dense(horizon)  # saída multi-step
])

# Compilação
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
    loss="mse",
    metrics=["mae"]
)

callbacks = [
    EarlyStopping(monitor="val_loss", patience=200, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=200, min_lr=1e-5, verbose=1)
]

history = model.fit(
    X_train, y_train,
    epochs=2000,
    batch_size=16,          # batch maior estabiliza gradiente
    validation_data = (X_test, y_test),
    callbacks=callbacks,
    verbose=1
)

In [ ]:
y_pred = model.predict(X_test)

y_test_inv = scaler.inverse_transform(y_test)
y_pred_inv = scaler.inverse_transform(y_pred)

mae_lstm_multi = mean_absolute_error(y_test_inv.flatten(), y_pred_inv.flatten())
print('LSTM Multi-step MAE:', mae_lstm_multi)

In [ ]:
fig = plota_resultados(
    df=mmm_marinha,
    x_col="ano_mes",
    y_col="consumo",
    title="Previsão temporal — Multi-step LSTM",
    preds={
        "Multi-step LSTM": (test.ano_mes, y_pred_inv[-1])
    }
)
fig.show()

#### N-Beats

In [ ]:
train['item_id'] = 'mnc'
nbeats_data = train.rename(columns={'item_id':'unique_id', 'ano_mes':'ds', 'consumo':'y'})

model = NBEATS(
    h = 12,
    input_size = 36,
    stack_types = ['seasonality', 'identity', 'trend', 'identity'],
    n_blocks = [3, 3, 3, 2],
    activation = 'ReLU',

    learning_rate = 1e-3,
    num_lr_decays = 3,
    batch_size = 16,
    scaler_type = 'robust',


    max_steps = 1000,
    val_check_steps = 10,
    early_stop_patience_steps = 20
)

nbeats_forecast = NeuralForecast(models=[model], freq='M')
nbeats_forecast.fit(df=nbeats_data, val_size=12)
y_pred_nbeats = nbeats_forecast.predict()

In [ ]:
fig = plota_resultados(
    df=mmm_marinha,
    x_col="ano_mes",
    y_col="consumo",
    title="Previsão temporal — N-Beats",
    preds={
        "N-Beats": (test.ano_mes, y_pred_nbeats.NBEATS.values.flatten())
    }
)
fig.show()

#### Autogluon

In [ ]:
autogluon_data = nbeats_data.rename(columns={'unique_id':'item_id', 'ds':'timestamp', 'y':'target'})

prediction_length = 12

train_df = autogluon_data.iloc[:-prediction_length].copy()
test_df  = autogluon_data.iloc[-prediction_length:].copy()

val_size = prediction_length  # validação com mesmo horizonte
train_for_fit = autogluon_data.iloc[:-val_size].copy()
tune_for_fit = autogluon_data.iloc[-val_size - prediction_length : -prediction_length].copy()  # segmento anterior ao teste

predictor = TimeSeriesPredictor(
    target='target',
    prediction_length=prediction_length,
    eval_metric='MAE'
)

predictor.fit(
    train_data=train_for_fit,
    presets='best_quality',
    time_limit=None,
    verbosity=2
)

# Simples: usamos o conjunto completo (AutoGluon deduz o ponto de corte pelo prediction_length)
predictions = predictor.predict(autogluon_data)

In [ ]:
fig = plota_resultados(
    df=mmm_marinha,
    x_col="ano_mes",
    y_col="consumo",
    title="Previsão temporal — Autogluon",
    preds={
        "Autogluon": (test.ano_mes, predictions['mean'].values)
    }
)
fig.show()

### Seleção do melhor modelo

In [ ]:
# #@title Comparação de modelos
# import ipywidgets as widgets
# from IPython.display import display, clear_output

# # Dropdowns interativos
# model_dropdown = widgets.Dropdown(
#     options=["SARIMAX", "Prophet", "XGBoost", "ExponentialSmoothing", "LSTM", "N-BEATS", "AutoGluon"],
#     value="LSTM",
#     description="Modelo:"
# )

# test_dropdown = widgets.Dropdown(
#     options=["A", "B", "C"],
#     value="A",
#     description="Teste:"
# )

# output = widgets.Output()

# def update_plot(change):
#     with output:
#         clear_output()
#         model = model_dropdown.value
#         test_set = test_dropdown.value

#         print(f"📊 Modelo selecionado: {model}")
#         print(f"📂 Conjunto de teste : {test_set}")

#         # Seleção de previsão
#         if model == "LSTM":
#             y_pred = pred_lstm
#         elif model == "XGBoost":
#             y_pred = pred_xgb
#         elif model == "Prophet":
#             y_pred = forecast_test['yhat']
#         elif model == "SARIMAX":
#             y_pred = pred_sarimax
#         elif model == "ExponentialSmoothing":
#             y_pred = pred_hw
#         elif model == "N-BEATS":
#             y_pred = y_pred_nbeats
#         elif model == "AutoGluon":
#             y_pred = y_pred_autogluon

#         # Plot interativo
#         fig = grafico_base(f"Previsão temporal com {model}")
#         fig.add_scatter(x=test.ano_mes, y=y_pred, mode="lines+markers", name=f"{model} Forecast")
#         fig.show()

# # Ligando evento
# model_dropdown.observe(update_plot, names="value")
# test_dropdown.observe(update_plot, names="value")

# display(model_dropdown, test_dropdown, output)

# # Render inicial
# update_plot(None)


### Conclusões

## Trabalhos futuros